# Initialize the accelerator

In [1]:
from finn_examples import models
print(list(filter(lambda x: "imagenet" in x, dir(models))))

['_imagenet_top5inds_io_shape_dict', 'mobilenetv1_w4a4_imagenet']


In [2]:
#pynq.Device.active_device.name does not correctly resolve target platform automatically (for "ZCU102")
#accel = models.mobilenetv1_w4a4_imagenet()
#accel = models.mobilenetv1_w4a4_imagenet("ZCU102")

from finn_examples import driver
accel = driver.FINNExampleOverlay("/home/xilinx/mobilenetv1-w4a4/bitfile/finn-accel.bit", 
                                  "zynq-iodma", 
                                  models._imagenet_top5inds_io_shape_dict,
                                  batch_size=1, fclk_mhz=100.0)

In [3]:
print("Expected input shape and datatype: %s %s" % (str(accel.ishape_normal), str(accel.idt)))
print("Expected output shape and datatype: %s %s" % (str(accel.oshape_normal), str(accel.odt)))

Expected input shape and datatype: (1, 224, 224, 3) DataType.UINT8
Expected output shape and datatype: (1, 1, 1, 5) DataType.UINT16


# Load the ImageNet validation dataset

In [4]:
import numpy as np
from PIL import Image
from dataset_loading import FileQueue, ImgQueue

#import os
#valdir = os.environ["IMAGENET_VAL_PATH"]

valdir = "/home/xilinx/dataset/ILSVRC2012_img_val/"

In [5]:
def img_resize(img, size):
    w, h = img.size
    if (w <= h and w == size) or (h <= w and h == size):
        return img
    if w < h:
        ow = size
        oh = int(size * h / w)
        return img.resize((ow, oh), Image.BILINEAR)
    else:
        oh = size
        ow = int(size * w / h)
        return img.resize((ow, oh), Image.BILINEAR)

def img_center_crop(img, size):
    crop_height, crop_width = (size, size)
    image_width, image_height = img.size

    crop_top = int(round((image_height - crop_height) / 2.))
    crop_left = int(round((image_width - crop_width) / 2.))

    return img.crop((crop_left, crop_top, crop_left + crop_width, crop_top + crop_height))

def pre_process(img_np):
    img = Image.fromarray(img_np.astype(np.uint8))
    img = img_resize(img, 256)
    img = img_center_crop(img, 224)
    img = np.array(img, dtype=np.uint8)
    return img

In [6]:
batch_size=1
files = ['ILSVRC2012_val_{:08d}.JPEG'.format(i) for i in range(1,50001)]
labels = np.loadtxt(valdir + "val.txt", dtype=int, usecols=1)
file_queue = FileQueue()
file_queue.load_epochs(list(zip(files,labels)), shuffle=False)
img_queue = ImgQueue(maxsize=batch_size)
img_queue.start_loaders(file_queue, num_threads=1, img_dir=valdir, transform=pre_process)

# Classify a single image

In [7]:
from matplotlib import pyplot as plt

test_single_x, test_single_y = img_queue.get()

# cleanup data loader
file_queue.join()
img_queue.join()

plt.imshow(test_single_x)
plt.show()

In [8]:
test_single_y

65

In [ ]:
accel_in = test_single_x.reshape(accel.ishape_normal)
print("Input buffer shape is %s and datatype is %s" % (str(accel_in.shape), str(accel_in.dtype)))

Input buffer shape is (1, 224, 224, 3) and datatype is uint8


In [ ]:
accel_out = accel.execute(accel_in)

In [ ]:
print("Top-5 classes predicted by the accelerator: " + str(accel_out))

In [ ]:
%%timeit
accel_out = accel.execute(accel_in)

# Validate accuracy on entire ImageNet validation set

In [ ]:
batch_size = 100
accel.batch_size = batch_size
print("Accelerator buffer shapes are %s for input, %s for output" % (str(accel.ishape_packed), str(accel.oshape_packed)) )
obuf_packed = np.empty_like(accel.obuf_packed_device)

files = ['ILSVRC2012_val_{:08d}.JPEG'.format(i) for i in range(1,50001)]
labels = np.loadtxt(valdir + "val.txt", dtype=int, usecols=1)
file_queue = FileQueue()
file_queue.load_epochs(list(zip(files,labels)), shuffle=False)
img_queue = ImgQueue(maxsize=batch_size)
img_queue.start_loaders(file_queue, num_threads=4, img_dir=valdir, transform=pre_process)

In [ ]:
ok = 0
nok = 0
i = 0
while not img_queue.last_batch:

    imgs, lbls = img_queue.get_batch(batch_size, timeout=None)
    imgs = np.stack(imgs)
    exp = np.array(lbls)
    
    ibuf_normal = imgs.reshape(accel.ishape_normal)
    obuf_normal = accel.execute(ibuf_normal)
    obuf_normal = obuf_normal.reshape(batch_size, -1)[:,0]
    ret = np.bincount(obuf_normal.flatten() == exp.flatten())
    nok += ret[0]
    ok += ret[1]
    i += 1
    print("batch %d : total OK %d NOK %d" % (i, ok, nok))

In [ ]:
total = 50000
acc = 100.0 * ok / (total)
print("Final top-1 accuracy: {}%".format(acc))

## More benchmarking

In [ ]:
from finn_examples import models
from finn_examples import driver

accel = driver.FINNExampleOverlay("/home/xilinx/mobilenetv1-w4a4/bitfile/finn-accel.bit", 
                                  "zynq-iodma", 
                                  models._imagenet_top5inds_io_shape_dict,
                                  batch_size=100, fclk_mhz=100.0)

accel.throughput_test()